# Exploratory data analysis

In [61]:
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

import pandas as pd
pd.options.display.max_columns = 10
pd.options.display.max_rows = 10

import warnings
warnings.filterwarnings('ignore')

import numpy as np

In [2]:
from chatto_transform.sessions.mimic import mimic_common

### You can also enter arbitrary SQL queries:

### Type in the following SQL query in the text area that appears:

````
SELECT * FROM MIMICIII.ADMISSIONS LIMIT 3
````

### Then click *Execute*.

Pull out all heart patients:
<pre>
SELECT subject_id, hadm_id, transfertime
FROM MIMICIII.SERVICES
WHERE curr_service = 'CMED'
</pre>

Find item IDs for potassium labs:
<pre>
SELECT *
FROM MIMICIII.D_LABITEMS
WHERE label ILIKE '%potassium%'
</pre>

Find item IDs for heart items:
<pre>
SELECT *
FROM MIMICIII.D_ITEMS
WHERE label ILIKE '%heart%'
</pre>

Pull MAPs:
<pre>
SELECT *
FROM MIMICIII.D_ITEMS
WHERE label ILIKE 'art map'
    OR label ILIKE 'fem map'
    OR label ILIKE 'FEM ALINE MAP'
    OR label ILIKE 'FEM ART MAP'
    OR label ILIKE 'FEM LINE MAP'
    OR label ILIKE 'High MAP ALARM'
    OR label ILIKE 'IR Sheath MAP'
    OR label ILIKE 'Low MAP ALARM'
    OR label ILIKE 'MAP'
    OR label ILIKE 'R FEM MAP'
    OR label ILIKE 'RADIAL MAP'
    OR label ILIKE 'LEFT RADIAL MAP'
    OR label ILIKE 'SHEATH MAP' 
</pre>

Pull all potassium labs:
<pre>
SELECT * 
FROM MIMICIII.LABEVENTS
WHERE itemid = 50971
LIMIT 10
<pre>

In [3]:
mimic_common.sql()

Loaded SELECT * 
FROM MIMICIII.LABEVENTS
WHERE itemid = 50971
LIMIT 10
 and stored in loaded_sql[0]
Loaded SELECT cmed.subject_id, cmed.hadm_id AS cmed_hadm_id, cmed.transfertime AS cmed_transfertime
FROM
(SELECT subject_id, hadm_id, transfertime
    FROM 
    mimiciii.services
    WHERE curr_service = 'CMED'
    LIMIT 10
) AS cmed and stored in loaded_sql[1]
Loaded SELECT cmed.subject_id, cmed.hadm_id AS cmed_hadm_id, cmed.transfertime AS cmed_transfertime
FROM
(SELECT subject_id, hadm_id, transfertime
    FROM 
    mimiciii.services
    WHERE curr_service = 'CMED'
    LIMIT 10) AS cmed
LEFT JOIN mimiciii.labevents AS lab
    ON lab.subject_id = cmed.subject_id
 and stored in loaded_sql[2]
Loaded SELECT cmed.subject_id, cmed.hadm_id AS cmed_hadm_id, cmed.transfertime AS cmed_transfertime,
    lab.charttime AS k_charttime, lab.value AS k_value, lab.valuenum AS k_valuenum, lab.uom AS k_uom, lab.hadm_id AS k_hadm_id, lab.itemid AS k_itemid
FROM
(SELECT subject_id, hadm_id, transfertime
 

### And access the results like this:

Find K+ labs for cardiac medicine patients:
* find cmed pts
* join with K+ labs

<pre>
SELECT cmed.subject_id, cmed.hadm_id AS cmed_hadm_id, cmed.transfertime AS cmed_transfertime,
    lab.charttime AS k_charttime, lab.value AS k_value, lab.valuenum AS k_valuenum, lab.uom AS k_uom, lab.hadm_id AS k_hadm_id, lab.itemid AS k_itemid
FROM
(SELECT subject_id, hadm_id, transfertime
    FROM 
    mimiciii.services
    WHERE curr_service = 'CMED') AS cmed
LEFT JOIN mimiciii.labevents AS lab
    ON lab.subject_id = cmed.subject_id
    AND lab.itemid = 50971
</pre>

In [46]:
first10

,subject_id,cmed_hadm_id,cmed_transfertime,k_charttime,k_value,k_valuenum,k_uom,k_hadm_id,k_itemid
0,154,102354,2127-12-23 18:47:00,2127-12-23 19:50:00,11,11.00,mEq/L,102354,50868
1,154,102354,2127-12-23 18:47:00,2127-12-23 19:50:00,21,21.00,mEq/L,102354,50882
2,154,102354,2127-12-23 18:47:00,2127-12-23 19:50:00,110,110.00,mEq/L,102354,50902
3,154,102354,2127-12-23 18:47:00,2127-12-23 19:50:00,84,84.00,IU/L,102354,50910
4,154,102354,2127-12-23 18:47:00,2127-12-23 19:50:00,0.9,0.90,mg/dL,102354,50912
...,...,...,...,...,...,...,...,...,...
6951,194,124794,2112-05-23 10:27:00,2112-10-15 07:10:00,88,88.00,fL,NaN,51250
6952,194,124794,2112-05-23 10:27:00,2112-10-15 07:10:00,313,313.00,K/uL,NaN,51265
6953,194,124794,2112-05-23 10:27:00,2112-10-15 07:10:00,14.5,14.50,%,NaN,51277
6954,194,124794,2112-05-23 10:27:00,2112-10-15 07:10:00,4.69,4.69,m/uL,NaN,51279


In [48]:
first10.dtypes

subject_id              int64
cmed_hadm_id            int64
cmed_transfertime    category
k_charttime          category
k_value              category
k_valuenum            float64
k_uom                category
k_hadm_id             float64
k_itemid                int64
dtype: object

In [57]:
# Convert pandas DF dates to days between transfer to CMED and potassium lab value.
transf_time = pd.to_datetime(first10['cmed_transfertime'])
potass_time = pd.to_datetime(first10['k_charttime'])
d_to_k = potass_time - transf_time
d_to_k / np.timedelta64(1,'us') / (24*3600*1e6)

In [45]:
sql_res = mimic_common.loaded_sql[9]
print(sql_res[:10][['itemid','label']])
print(sql_res[10:][['itemid','label']])

   itemid           label
0     438             MAP
1    1200  High MAP Alarm
2    1201   Low MAP alarm
3    1321             map
4    3067         FEM MAP
5    5680      RADIAL MAP
6    5804      sheath MAP
7    2974    FEM LINE MAP
8    2544     FEM ART MAP
9    2309   FEM ALINE MAP
    itemid            label
10    2353        R FEM MAP
11    2369          fem map
12    2770  left radial MAP
13    6399          ART MAP
14    6579    IR Sheath MAP


MAP items in carevue:
* 438 (MAP)
* 1321 (map)

<pre>
   itemid           label
0     438             MAP
1    1200  High MAP Alarm
2    1201   Low MAP alarm
3    1321             map
4    3067         FEM MAP
5    5680      RADIAL MAP
6    5804      sheath MAP
7    2974    FEM LINE MAP
8    2544     FEM ART MAP
9    2309   FEM ALINE MAP
    itemid            label
10    2353        R FEM MAP
11    2369          fem map
12    2770  left radial MAP
13    6399          ART MAP
14    6579    IR Sheath MAP
</pre>

In [10]:
first10 = mimic_common.loaded_sql[3]
heart_items = mimic_common.loaded_sql[4]

In [15]:
heart_items.dtypes

row_id                int64
itemid                int64
label              category
abbreviation       category
dbsource           category
                     ...   
category           category
unitname           category
param_type         category
lownormalvalue      float64
highnormalvalue     float64
dtype: object

In [16]:
heart_items

,row_id,itemid,label,abbreviation,dbsource,...,category,unitname,param_type,lownormalvalue,highnormalvalue
0,212,211,Heart Rate,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
1,213,212,Heart Rhythm,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
2,214,213,Heart Sounds,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
3,2064,5972,fetal heartrate,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
4,1078,3494,Lowest Heart Rate,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29,12765,220045,Heart Rate,HR,metavision,...,Routine Vital Signs,bpm,Numeric,NaN,NaN
30,12766,220046,Heart rate Alarm - High,HR Alarm - High,metavision,...,Alarms,bpm,Numeric,NaN,NaN
31,12767,220047,Heart Rate Alarm - Low,HR Alarm - Low,metavision,...,Alarms,bpm,Numeric,NaN,NaN
32,12768,220048,Heart Rhythm,Heart Rhythm,metavision,...,Routine Vital Signs,NaN,Text,NaN,NaN


In [18]:
carevue_heart = heart_items[heart_items.dbsource=='carevue']
metavision_heart = heart_items[heart_items.dbsource=='metavision']

In [19]:
carevue_heart

,row_id,itemid,label,abbreviation,dbsource,...,category,unitname,param_type,lownormalvalue,highnormalvalue
0,212,211,Heart Rate,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
1,213,212,Heart Rhythm,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
2,214,213,Heart Sounds,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
3,2064,5972,fetal heartrate,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
4,1078,3494,Lowest Heart Rate,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN
5,3786,2781,fetal heart sound,NaN,carevue,...,NaN,NaN,NaN,NaN,NaN


From carevue, we want item 211 (heart rate).

From metavision, we want items 220045 (heart rate) and 220046 (heart rate alarm - high).

In [24]:
metavision_heart[:10]

,row_id,itemid,label,abbreviation,dbsource,...,category,unitname,param_type,lownormalvalue,highnormalvalue
6,15062,228189,CO-Tandem Heart Flow,CO-Tandem Heart Flow,metavision,...,Tandem Heart,L/min,Numeric,NaN,NaN
7,15066,228194,Peripheral Pulses (Tandem Heart),Peripheral Pulses (Tandem Heart),metavision,...,Tandem Heart,NaN,Text,NaN,NaN
8,15068,228198,Tandem Heart Flow,Tandem Heart Flow,metavision,...,Tandem Heart,L/min,Numeric,NaN,NaN
9,15069,228199,Type (Tandem Heart),Type (Tandem Heart),metavision,...,Tandem Heart,NaN,Text,NaN,NaN
10,15070,228201,Tandem Heart Access Line,Tandem Heart Access Line,metavision,...,Access Lines - Invasive,None,Process,NaN,NaN
11,15071,228202,Tandem Heart Return Line,Tandem Heart Return Line,metavision,...,Access Lines - Invasive,None,Process,NaN,NaN
12,15072,228203,Tandem Heart Access Line Cannula Site,Tandem Heart Access Line Cannula Site,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
13,15073,228204,Tandem Heart Access Line Catheter Length,Tandem Heart Access Line Catheter Length,metavision,...,Access Lines - Invasive,cm,Numeric,NaN,NaN
14,15074,228206,Tandem Heart Access Line Discontinued,Tandem Heart Access Line Discontinued,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
15,15075,228207,Tandem Heart Access Line Dressing Change,Tandem Heart Access Line Dressing Change,metavision,...,Access Lines - Invasive,NaN,Date time,NaN,NaN


In [25]:
metavision_heart[10:20]

,row_id,itemid,label,abbreviation,dbsource,...,category,unitname,param_type,lownormalvalue,highnormalvalue
16,15076,228209,Tandem Heart Access Line Insertion Date,Tandem Heart Access Line Insertion Date,metavision,...,Access Lines - Invasive,NaN,Date time,NaN,NaN
17,15077,228210,Tandem Heart Access Line Position,Tandem Heart Access Line Position,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
18,15078,228211,Tandem Heart Access Line Postion Confirmed,Tandem Heart Access Line Position Confirmed,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
19,15079,228214,Tandem Heart Access Line Site Appear,Tandem Heart Access Line Site Appear,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
20,15080,228215,Tandem Heart Access Line Tubing Change,Tandem Heart Access Line Tubing Change,metavision,...,Access Lines - Invasive,NaN,Date time,NaN,NaN
21,15081,228216,Tandem Heart Access Line Type,Tandem Heart Access Line Type,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
22,15082,228217,Tandem Heart Return Cannula Discontinued,Tandem Heart Return Cannula Discontinued,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
23,15083,228218,Tandem Heart Return Cannula Dressing Change,Tandem Heart Return Cannula Dressing Change,metavision,...,Access Lines - Invasive,NaN,Date time,NaN,NaN
24,15084,228221,Tandem Heart Return Cannula Insertion Date,Tandem Heart Return Cannula Insertion Date,metavision,...,Access Lines - Invasive,NaN,Date time,NaN,NaN
25,15085,228223,Tandem Heart Return Cannula Position,Tandem Heart Return Cannula Position,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN


In [26]:
metavision_heart[20:]

,row_id,itemid,label,abbreviation,dbsource,...,category,unitname,param_type,lownormalvalue,highnormalvalue
26,15086,228225,Tandem Heart Return Cannula Site,Tandem Heart Return Cannula Site,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
27,15087,228226,Tandem Heart Return Cannula Site Appear,Tandem Heart Return Cannula Site Appear,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
28,15088,228227,Tandem Heart Return Cannula Size,Tandem Heart Return Cannula Size,metavision,...,Access Lines - Invasive,NaN,Text,NaN,NaN
29,12765,220045,Heart Rate,HR,metavision,...,Routine Vital Signs,bpm,Numeric,NaN,NaN
30,12766,220046,Heart rate Alarm - High,HR Alarm - High,metavision,...,Alarms,bpm,Numeric,NaN,NaN
31,12767,220047,Heart Rate Alarm - Low,HR Alarm - Low,metavision,...,Alarms,bpm,Numeric,NaN,NaN
32,12768,220048,Heart Rhythm,Heart Rhythm,metavision,...,Routine Vital Signs,NaN,Text,NaN,NaN
33,13432,224389,Heart Sounds,Heart Sounds,metavision,...,Cardiovascular,NaN,Text,NaN,NaN


In [6]:
cmed_pts = mimic_common.loaded_sql[1]

In [7]:
cmed_pts

,subject_id,hadm_id,transfertime
0,154,102354,2127-12-23 18:47:00
1,154,102354,2127-12-24 00:47:40
2,160,161672,2174-11-06 08:38:58
3,163,138528,2146-06-21 17:42:45
4,164,182743,2116-12-28 15:34:05
...,...,...,...
9130,92287,133462,2171-02-16 18:12:20
9131,92292,153237,2163-12-28 08:00:16
9132,92295,111545,2181-06-08 20:22:24
9133,92316,158581,2187-05-05 18:02:51


In [17]:
# Number of unique patients on CMED service
cmed_pts.subject_id.nunique()

7949

In [21]:
# Potassium lab codes
potassium_labs = mimic_common.loaded_sql[6]
potassium_labs

,row_id,itemid,label,fluid,category,loinc_code
0,264,51064,"POTASSIUM, STOOL",STOOL,CHEMISTRY,15202-5
1,297,51097,"POTASSIUM, URINE",URINE,CHEMISTRY,2828-2
2,23,50822,"POTASSIUM, WHOLE BLOOD",BLOOD,BLOOD GAS,6298-4
3,34,50833,POTASSIUM,OTHER BODY FLUID,BLOOD GAS,2821-7
4,48,50847,"POTASSIUM, ASCITES",ASCITES,CHEMISTRY,49789-1
5,172,50971,POTASSIUM,BLOOD,CHEMISTRY,2823-3
6,241,51041,"POTASSIUM, BODY FLUID",OTHER BODY FLUID,CHEMISTRY,2821-7
7,257,51057,"POTASSIUM, PLEURAL",PLEURAL,CHEMISTRY,NaN


We want itemid = 50971 (blood chemistry).

### Let's get into doing stuff with code. You can load tables and queries without using the visual widgets above:

In [ ]:
from chatto_transform.schema.mimic import mimic_schema

In [ ]:
df = mimic_common.load_table(mimic_schema.admissions_schema)

In [ ]:
df

### We just loaded the `admissions` table.

### We did this by called `mimic_common.load_table`, and passing it a `schema` object.

### Let's take a closer look at the `schema` we used, `mimic_schema.admissions_schema`:

In [ ]:
mimic_schema.admissions_schema

### Here we see information about the `admissions` table. We see the names of types of the different columns.

### We can pass any of the schema objects in `mimic_schema` to `load_table`, and the table will be loaded. Feel free to take a look at the different schemas:

In [ ]:
dir(mimic_schema)

### Say we want to download the table we just loaded. We can do it like this:

In [ ]:
mimic_common.df_to_csv('admissions.csv', df)

# First pass: Select potassium labs  

In [ ]:
from chatto_transform.sessions.mimic import cohorts

In [ ]:
my_cohort = cohorts.Cohort()

In [ ]:
my_cohort.filter_labevents()

In [ ]:
my_cohort.summary_fields

In [ ]:
my_cohort.summary()

In [ ]:
x = 3